## Background Information 
When I developed the proof-of-concept for Part 2 (Progress Report for this project), I followed the standards that are used for implementation of most clustering processes: read in all data points (aka pixels) as data stream and process the information based on the RGB value for each pixel (one after the other). 

This approach allowed me to achieve successful KMEANS clustering for images, though it was very slow (run times up to several minutes, in some cases hours)! 

Once I had completed the implementation of the Density Peak (DP) clustering algorithm, it became clear that I had reached a dead end with my code: not a single DP segmentation process could complete within hours of running! 
I started researching alternative Python libraries, but they all had the same problem: once faced with the processing of 100,000+ data points (aka pixels), we can't just stream the data any longer to perform basic calculations, such as: find the maximum possible distance between any two points in the set, or find the closest pixel for each pixel in the image. All these operations are of the order O(N^2) and cannot be completed by any program in reasonable timeframes. 

After hitting this dead end, I thought about alternative options and it became clear that two things would help to solve this issue: 
- Somehow using indexed data points to improve access to relevant data (pixels) without the need to loop through all data records (in relational database terms this is a 'full table scan' vs 'indexed data access') 
- meaningful reduction of data points for processing, without compromising on the quality of the image too much

I conceived the following two approaches to implement those two objectives: 

Ad 1: It was beyond the time and scope of this project to set up a relational database schema for the image pixels. However there is the concept of 'Dictionaries' available in Python. It uses a hash-key to access data for a particular key (or index) and offers excellent performance. I have decided to convert all list loops, eg to find an attribute for a particular pixel, into dictionaries. This offers significant improvements for the overall KMEANS and DP clustering algorithms. 

Ad 2: Rather than working with individual pixels, I have divided the RGB cube (with length 256) into several sub-cubes of equal length (e.g. cubes with length to the power of 2). For each mini-cube within the overall RGB cube, I have then counted the number of pixels that sit within the mini-cube, and assigned this number to the pixel in the centre of the mini-cube. Completing this process for an image with 100,000+ pixels, allowed me to compress the number of distinct data points (pixels) below 1,000 in most cases! 

With the 2 steps described above, the structure of the input data had been significantly changed, and the traditional KMEANS clustering process did no longer work. Code became far more complex (for example, the KMEANS implementation increased from about 50 lines to over 150 lines of code!) However, the results were astounding: KMEANS processes, that ran for 10 min and longer could now be completed in under 5 sec! 

The approach was a compromise: giving away a little bit of image accuracy allowed me to achieve very good runtimes for KMEANS and DP clustering algorithms. 

The following section outlines the process that had been used to determine the best value for the length of the mini-cubes (called granularity in this notebook). The optimal parameter for the mini-cube length was then used as the default setting in the GUI interface and the tuning of the hyper-parameters of the DP clustering algorithm. 


## RGB Cubes for Preprocessing of Images

We start by importing library matplotlib for inline display of images in this workbook. 

In [ ]:
# Display all images inline
%matplotlib inline
import matplotlib.pyplot as plt


Now we import code that has been developed for the slicing of the cubes. Library PIL is imported so we can process pixel stream between image objects.  


In [ ]:
from dp import DPImage
from PIL import Image

We test the image pre-processing with all possible side lengths for the mini cubes: 1,2,4,8,16,32,64,128 and 256.

Please note: 

- If length of cube is set to 1, then the quality of the image will not decrease: each pixel is assigned to itself (its own image cube, which is effectively just a point) 
- If length of the cube is set to 256, we are left with a single cube, the RGB space itself! Each pixel will be assigned to the centre of this space which is (128,128,128) and grey! 

This illustrates that the optimal value for the length of the cube most be somewhere between 1 and 256. I've tested the process against a beautiful picture of the Fox glacier (taken in January 2018 during holidays in New Zealand). 

We start by generating images with side length 1, 2, 4, 8, 16 and 32:

#### In the original image k.jpeg, there are 139,968 pixels. 


In [ ]:
img_name = "images/k.jpeg"
img1 = Image.open(img_name)
img2 = Image.open(img_name)

dist = 'Euclidean'    
dp = DPImage(dist) 

for granularity in [1,2,4,8,16,32]: 
    dp.GRANULARITY = granularity
    dp.pre_process_img(img1)
    img2.putdata(dp.get_pre_processed_data())
    
    f, axarr = plt.subplots(1,2)
    f.suptitle("Granularity = "+str(dp.GRANULARITY)+", Number of Distinct Pixels = "+str(len(dp.pnts.keys())), fontsize=16)
    axarr[0].set_title('Original')
    axarr[0].imshow(img1)
    axarr[0].axis('off')
    axarr[1].set_title('Clustered')
    axarr[1].imshow(img2)   
    axarr[1].axis('off') 
    plt.show() 

Now we test against the remaining side lengths (64, 128 and 256). The purpose of this section is more to cross-check that the implemented algorithm works correctly for all specificed side lengths, but the generated images lost too much of their details and should no longer be used in DP or KMEANS clustering: 

In [ ]:
img_name = "images/k.jpeg"
img1 = Image.open(img_name)
img2 = Image.open(img_name)

dist = 'Manhattan'    
dp = DPImage(dist) 

for granularity in [64,128,256]: 
    dp.GRANULARITY = granularity
    dp.pre_process_img(img1)
    img2.putdata(dp.get_pre_processed_data())
    
    f, axarr = plt.subplots(1,2)
    f.suptitle("Granularity = "+str(dp.GRANULARITY)+", Number of Distinct Pixels = "+str(len(dp.pnts.keys())), fontsize=16)
    axarr[0].set_title('Original')
    axarr[0].imshow(img1)
    axarr[0].axis('off')
    axarr[1].set_title('Clustered')
    axarr[1].imshow(img2)   
    axarr[1].axis('off') 
    plt.show() 

Note: with granularity = 1, there is no difference between the original and the pre=processed image. With granularity = 256, only one large 'cube' remains in the RGB space. Its centre is (128,128,128) and this is exactly the colour that is shown for the pre-processed image. 

However, it is remarkable to note that cube size 128 produces a total of 8 clusters (2^3), which in themselves could be considered as a reasonable segmentation of the initial image with a fix number of 8 segments! 

#### Result: we achieve reasonable image quality in the pre-processed image with GRANULARITY=16. We will use this as standard value for all image clustering processes. Please note: the value is configurable in the app.ini file, in the global section and may be altered.  

## Estimation of Average Distance between Pixels in Mini RGB-Cube
All pixels in a data cube are now centred in the centre of the cube. This means that the algorithm will calculate a distance of 0 for all data points in the cube. However, this is not correct and we use the estimated average distance between two points in a cube. 
Rather than mathematically calculating this mean, I have decided to run a simple simulation to estimate the mean differences for Euclidean, Manhattan and Supremum distances. Those values are then used in the density calculation for all pixels in the DP algorithm. 

In [ ]:
from random import random
from math   import sqrt
e=0.0
m=0.0
s=0.0
n=1000000
for i in range(n):
    e = e + sqrt(pow(random() - random(),2) + pow(random() - random(),2) + pow(random() - random(),2)) 
    m = m + abs(random() - random()) + abs(random() - random()) + abs(random() - random()) 
    s = s + max(abs(random() - random()),abs(random() - random()),abs(random() - random())) 
print("Average Manhattan Distance: "+str(m/n))
print("Average Euclidean Distance: "+str(e/n))
print("Average Supremum Distance:  "+str(s/n))

We will use the following mean values for distances between pixels in the same cube: 
- Manhattan Distance: 1.00
- Euclidean Distance: 0.66 
- Supremum Distance:  0.54

In [ ]:
e=0.0
m=0.0
s=0.0
n=1000000
for i in range(n):
    e = e + sqrt(pow(1+random() - random(),2) + pow(random() - random(),2) + pow(random() - random(),2)) 
    m = m + abs(1+random() - random()) + abs(random() - random()) + abs(random() - random()) 
    s = s + max(abs(1+random() - random()),abs(random() - random()),abs(random() - random())) 
print("Average Manhattan Distance: "+str(m/n))
print("Average Euclidean Distance: "+str(e/n))
print("Average Supremum Distance:  "+str(s/n))

In [ ]:
e=0.0
m=0.0
s=0.0
n=1000000
for i in range(n):
    e = e + sqrt(pow(2+random() - random(),2) + pow(random() - random(),2) + pow(random() - random(),2)) 
    m = m + abs(2+random() - random()) + abs(random() - random()) + abs(random() - random()) 
    s = s + max(abs(2+random() - random()),abs(random() - random()),abs(random() - random())) 
print("Average Manhattan Distance: "+str(m/n))
print("Average Euclidean Distance: "+str(e/n))
print("Average Supremum Distance:  "+str(s/n))

This concludes the workbook for the pre-processing of images in mini-cubes. 